In [2]:
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd
from IPython.core.display import display, HTML
import re
from re import sub
from decimal import Decimal
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from iso3166 import countries
import numpy as np

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [5]:
#pd.read_csv("Data_from_top_1000")

In [ ]:
url_search = "https://www.boxofficemojo.com/chart/never_in_top/?by_rank_threshold=1&offset=200"


In [ ]:
#driver = webdriver.Chrome(chromedriver)
driver.get(url_search)


In [28]:
movie_name_list = [] ###
budget_list = [] ###
MPAA_list = [] ###
run_time_list = [] ###
genre_list = [] ###
markets_list = [] ###
production_co_list = [] ###
release_month_list = [] ###
#director_list = []
no_of_releases = [] ###
in_theatres_list_duration = [] 
opening_list = [] ##
widest_release_list = [] 
world_revenue_list = [] ###

In [65]:
url ='https://www.boxofficemojo.com/chart/never_in_top/?by_rank_threshold=1&offset=200' 

response = requests.get(url)

page = response.text

soup = BeautifulSoup(page, "lxml")

ConnectionError: HTTPSConnectionPool(host='www.boxofficemojo.com', port=443): Max retries exceeded with url: /chart/never_in_top/?by_rank_threshold=1&offset=200 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbcee28cbe0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [95]:
movie_title_click = driver.find_elements_by_xpath(
    '//a[contains(@href, "/release")]')


count = 164
#for idx, movie in enumerate(movie_title_click):
while count < 1000:
    movie_title_click = driver.find_elements_by_xpath(
    '//a[contains(@href, "/release")]')
    movie_title_click[count].click()
    
    select_release = Select(driver.find_element_by_id("releasegroup-picker-navSelector"))
    select_release.select_by_visible_text("All Releases")
    
    
    #getting URL
    url = driver.current_url
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #Movie Title
    title =soup.find_all(name = "meta")
    title = str(title[3])
    title = title.split('"')[1]
    movie_name_list.append(title)
    
    #Budget
    if "Budget" not in page:
        budget_list.append("N/A")
    else:
        budget = soup.find_all("span", class_ = "money")
        if len(budget) < 5:
            budget = budget[2].text
            budget = float(Decimal(sub(r'[^\d.]', '', budget)))
            budget_list.append(budget)
        else:
            budget = budget[4].text
            budget = float(Decimal(sub(r'[^\d.]', '', budget)))
            budget_list.append(budget)
    
    #MPAA
    if "MPAA" not in page:
        MPAA_list.append("N/A")
    else:
        MPAA_regex = re.compile('MPAA')
        soup.find(text=MPAA_regex)
        MPAA_string = soup.find(text=re.compile('MPAA'))
        MPAA = MPAA_string.findNext()
        MPAA_list.append(MPAA)
    
    #RT
    if "Running Time" not in page:
        run_time_list.append("N/A")
    else:
        run_time = re.compile('Running Time')
        rt_string = soup.find(text=re.compile('Running Time'))
        rt = rt_string.findNext().text
        rt = rt.split()
        if len(rt) > 2:
            minutes = int(rt[0])*60 + int(rt[2])
            run_time_list.append(minutes)
        else:
            minutes = int(rt[0])*60
            run_time_list.append(minutes)
    
    #Genre
    if "Genres" not in page:
        genre_list.append("N/A")
    else:
        genre = re.compile('Genres')
        genre_string = soup.find(text=re.compile('Genres'))
        genre = genre_string.findNext().text
        genre = "\n".join(genre.split())
        genre = genre.replace("\n", ",").split()
        genre_list.append(genre)
    
    #Production C0
    if "Domestic Distributor" not in page:
        production_co_list.append("N/A")
    else:
        production_co = re.compile
        production_co = re.compile('Domestic Distributor')
        production_co = soup.find(text=re.compile('Domestic Distributor'))
        production_co = production_co.findNext().text
        production_co = (production_co.split("See")[0])
        production_co_list.append(production_co)
    
    #Release Month
    if "Earliest Release Date" not in page:
        release_month_list.append("N/A")
    else:
        release_month = re.compile
        release_month = re.compile('Earliest Release Date')
        release_month = soup.find(text=re.compile('Earliest Release Date'))
        release_month = release_month.findNext().text
        release_month = release_month.split(" ")[0]
        release_month_list.append(release_month)
    
    #Of Markets Released
    markets = 1
    tr_list = soup.find_all("tr")
    for i in tr_list:
        for c in countries:
            if c.name in i.text:
                markets += 1 
    no_of_markets = markets
    markets_list.append(no_of_markets)
    
    #No of Releases
    release_list =soup.find_all("select" , id = "releasegroup-picker-navSelector")
    releases = len((release_list[0].text).split("Re")) - 2
    no_of_releases.append(releases)
    
    #world revenue
    worldwide = soup.find_all("span", class_ = "money")
    worldwide = worldwide[2].text
    worldwide = float(Decimal(sub(r'[^\d.]', '', worldwide)))
    world_revenue_list.append(worldwide)
    
    driver.back()
    
    #navigating to other parts of the page for additional features
    #select_release = Select(driver.find_element_by_id("releasegroup-picker-navSelector"))
    #select_release.select_by_visible_text("Original Release")

    select_territory = Select(driver.find_element_by_xpath("//select[@name='release-picker-navSelector']"))
    select_territory.select_by_visible_text("Domestic")
    
    url = driver.current_url
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #Opening
    if "Opening" not in page:
        opening_list.append('N/A')
    else:
        opening = re.compile
        opening = re.compile('Opening')
        opening = soup.find(text=re.compile('Opening'))
        opening = opening.findNext().text
        o = opening.split("\n")
        o = o[0]  
        #o = float(Decimal(sub(r'[^\d.]', '', o)))
        opening_list.append(o)
    
    #in release
    if "In Release" not in page:
        release_monnth_list.append("N/A")
    else:
        in_release = re.compile
        in_release = re.compile('In Release')
        in_release = soup.find(text=re.compile('In Release'))
        in_release = in_release.findNext().text
        in_release = in_release.split(" ")
        in_release = (in_release[0])
    if "," in in_release:
            in_release = in_release.replace(",", "")
            in_release = int(in_release)
    else:
            in_release = int(in_release)
            in_theatres_list_duration.append(in_release)
    
    #wildest release
    if "Widest Release" not in page:
        widest_release_list.append("N/A")
    else:
        widest_release = re.compile
        widest_release = re.compile('Widest Release')
        widest_release = soup.find(text=re.compile('Widest Release'))
        widest_release = widest_release.findNext().text
        widest_release = widest_release.split(" ")
        widest_release = int(widest_release[0].replace(",",""))
        widest_release_list.append(widest_release)
    
    count+=1
    driver.back()
    #driver.back()
    #driver.back()
    #time.sleep(1)
    #going to next page
    if count == 200 or count == 399 or count == 599 or count == 799:
        next_page = driver.find_element_by_partial_link_text("Next page")
        next_page.click()
        driver.get(driver.current_url)
        movie_title_click = driver.find_elements_by_xpath(
        '//a[contains(@href, "/release/rl")]')
        count = 0
        
            
    if len(movie_name_list) == 1000:
        '''
        print(movie_name_list)
        print(budget_list)
        print(MPAA_list)
        print(run_time_list)
        print(genre_list)
        print(production_co_list)
        print(release_month_list)
        print(markets_list)
        print(no_of_releases)
        print(world_revenue_list)
        '''
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next page"}
  (Session info: chrome=94.0.4606.61)


In [96]:
print(len(movie_name_list))
print(len(budget_list))
print(len(MPAA_list))
print(len(run_time_list))
print(len(genre_list))
print(len(production_co_list))
print(len(release_month_list))
print(len(markets_list))
print(len(no_of_releases))
print(len(world_revenue_list))
print(len(opening_list))
print(len(in_theatres_list_duration))
print(len(widest_release_list))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [21]:
movie_title_click

[]

In [93]:
#opening_list.append("$18,237,568")
in_theatres_list_duration.append(365)
#widest_release_list.append(3018)
#in_theatres_list_duration

In [97]:
title

'Love, Simon'

In [98]:
data = { "Budget" : budget_list, "MPAA" : MPAA_list, "Run Time": run_time_list,
        "Genre" : genre_list, "No of Markets": markets_list, "Production Co": production_co_list,
        "Release Month": release_month_list, "Release Count": no_of_releases, 
         "Opening Revenue": opening_list,
        "Length in Theatres(Days)": in_theatres_list_duration, "Widest Release": widest_release_list,
        "World Revenue": world_revenue_list 
       }

In [99]:
df = pd.DataFrame(data, index = movie_name_list)

In [100]:
df

,Budget,MPAA,Run Time,Genre,No of Markets,Production Co,Release Month,Release Count,Opening Revenue,Length in Theatres(Days),Widest Release,World Revenue
Sing,75000000.0,[PG],108,"[Animation,Comedy,Family,Musical]",1,N/A,N/A,3,"$35,258,1454,022",376,4029,634151679.0
Sing,75000000.0,[PG],108,"[Animation,Comedy,Family,Musical]",1,N/A,N/A,3,"$35,258,1454,022",376,4029,634151679.0
Sing,75000000.0,[PG],108,"[Animation,Comedy,Family,Musical]",1,N/A,N/A,3,"$35,258,1454,022",376,4029,634151679.0
My Big Fat Greek Wedding,5000000.0,[PG],95,"[Comedy,Drama,Romance]",1,N/A,N/A,1,"$597,362108",622,2016,368744044.0
Alvin and the Chipmunks: The Squeakquel,75000000.0,[PG],88,"[Adventure,Animation,Comedy,Family,Fantasy,Mus...",1,N/A,N/A,1,"$48,875,4153,700",374,3747,443140005.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Revenge of the Nerds,N/A,N/A,90,[Comedy],1,Twentieth Century Fox,July,1,"$1,513,090364",165,989,1513090.0
Little,20000000.0,[PG-13],109,"[Comedy,Fantasy,Romance]",17,Universal Pictures,April,2,"$15,405,4552,667",266,2667,49000667.0
The Star,20000000.0,[PG],86,"[Adventure,Animation,Comedy,Family,Fantasy]",32,Sony Pictures Entertainment (SPE),November,3,"$9,812,6742,837",410,2976,62856743.0
Brewster's Millions,N/A,N/A,102,[Comedy],1,Universal Pictures,May,1,"$9,858,9051,521",224,1521,45833132.0


In [101]:
df.to_csv("Second_DF")

In [102]:
driver.quit()